In [3]:
from pyspark import SparkConf, SparkContext
from operator import itemgetter
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))


In [7]:
def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))

def path_step(item):
    prev_v, path, next_v  = item[0], item[1][0], item[1][1]
    prev_v = next_v
    path += tuple([next_v])
    return (prev_v , path, len(path))

def complete(item):
  v, old_d, new_d = item[0], item[1][0], item[1][1]
  return (v, old_d if old_d is not None else new_d)

In [8]:
n = 4  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

first_vertex = 12
second_vertex = 34

paths = sc.parallelize([(first_vertex , tuple([first_vertex]))]).partitionBy(n)

In [9]:
while True:
    paths = paths.join(forward_edges, n).map(path_step)
    if paths.filter(lambda x: x[0] == second_vertex).count() > 0:
        break
        
paths = paths.filter(lambda x: x[0] == second_vertex).collect()
path = min(paths, key=itemgetter(2))[1]

In [10]:
print(','.join(map(str, path)))

12,422,53,52,107,20,23,274,34
